## Data Modeling (03) : Logistic Regression

**Import Libraries**

In [1]:
# standards
import pandas as pd
import numpy as np
from pprint import pprint
from sara import eda, eda_unique

# modeling
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# nlp
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import regex as re

# viz
import seaborn as sns

# stop future warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

**Read in Processed Data**

In [2]:
df_reddit_process = pd.read_csv('data/reddit_processed.csv')
print(df_reddit_process.shape)

(110378, 8)


**Check for NaNs**

In [3]:
df_reddit_process['title'].isna().sum()

625

**Drop NaNs**

In [4]:
df_reddit_process.dropna(subset=['title'],inplace=True)

**Explore Balance of Subreddits**

In [5]:
df_reddit_process['subreddit'].value_counts()

1    93393
0    16360
Name: subreddit, dtype: int64

**Select X and Y Features**

In [6]:
X = df_reddit_process['title']
y = df_reddit_process['subreddit']

print('y nulls: ', y.isna().sum())
print('X nulls: ', X.isna().sum())

y nulls:  0
X nulls:  0


**Train Test Split**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    random_state=42)

**NLP MODEL FUCNTION**

In [ ]:
def nlp_model(X_train, X_test, y_train, y_test,
              transform, model, params):

    # STEP 1: Select Transform
    if transform == 'cvec':
        nlp_transform = 'cvec', CountVectorizer()
    if transform == 'tvec':
        nlp_transform = 'tvec', TfidfVectorizer()
    else:
        pass
    
    # STEP 2: Select Estimator
    if model == 'lr':
        nlp_model = 'lr', LogisticRegression()
    else:
        pass
    
    # STEP 3: Build Pipeline
    pipe = Pipeline([
        (nlp_transform),
        (nlp_model)
    ])
    
    # STEP 4: Run Model
    gs = GridSearchCV(pipe,
                      param_grid=params,
                      n_jobs=-1,
                      cv=5)
    gs.fit(X_train,y_train)
    train_score = gs.score(X_train,y_train)
    test_score = gs.score(X_test,y_test)
    model = gs
    
    # STEP 5: Extract Parameters as Dictionary
    best_parameters = gs.best_estimator_.get_params()
    param_dict = {}
    for param_name in params.keys():
        new_param = {
            param_name : best_parameters[param_name],
        }
        param_dict.update(new_param)
    
    # STEP 6: Extract Coefficients as Dictionary
    if model == 'lr':
        if transform == 'cvec':
            coef_values = gs.best_estimator_.named_steps.lr.coef_  
            coef_keys = gs.best_estimator_.named_steps.cvec.get_feature_names()
            coef_dict = {k : v for (k,v) in zip(coef_keys,list(coef_values[0]))}
        if transform == 'tvec':
            coef_values = np.exp(gs.best_estimator_.named_steps.lr.coef_  )
            coef_keys = gs.best_estimator_.named_steps.tvec.get_feature_names()
            coef_dict = {k : v for (k,v) in zip(coef_keys,list(coef_values[0]))} 
    else:
        coef_dict = None
    
    # STEP 7: Score Dictionary
    score_values = [model,transform,
              train_score,test_score,param_dict,coef_dict]
    score_keys = ['Model','Transform',
            'Train Score','Test Score','Parameters','Coefficients']
    score_dict = dict(zip(score_keys,score_values))
    
    # STEP 8: Score DataFrame
    data = {'Model': [model], 'Transform': [transform],
            'Train Score': [train_score], 'Test Score': [test_score]}
    df_new_row = pd.DataFrame(data)

    # STEP 9: update scores csv with new model
    df_old_scores = pd.read_csv('./data/all_scores.csv')
    merge_scores = [df_old_scores, df_new_row]
    df_all_scores = pd.concat(merge_scores, axis=0)
    df_all_scores.to_csv('./data/all_scores.csv', index=False)
    
    # STEP 10: Return Score Dictionary
    return df_new_row, score_dict, model

## Run Models

**Instantiate Model Score List**

**Model 1**

In [9]:
# Params 1
params1 = {
    'cvec__max_features' : [1000,5000,10000,20000],
    'cvec__ngram_range'  : [(1,1),(1,3),(1,5)]
}

# Model 1
model1 = nlp_model(X_train,X_test,y_train,y_test,'cvec','lr',params1)

**Model 2**

In [10]:
# Params 2
params2 = {  
    'tvec__max_features' : [1000,5000,10000],
    'tvec__ngram_range'  : [(1,1),(1,2),(1,3)]
}

# Model 2
model2 = nlp_model(X_train,X_test,y_train,y_test,'tvec','lr',params2)

**Model 3**

In [11]:
# Params 3
params3 = {
    'cvec__max_features' : [20_000],
    'cvec__ngram_range'  : [(1,5)]
}

# Model 3
model3 = nlp_model(X_train,X_test,y_train,y_test,'cvec','lr',params3)

**Model 4**

In [12]:
# Params 4
params4 = {
    'tvec__max_features' : [10000,15000],
    'tvec__ngram_range'  : [(1,3),(1,4)]
}

model4 = nlp_model(X_train,X_test,y_train,y_test,'tvec','lr',params4)

**Model 5**

In [13]:
# Params 5
params5 = {
    'cvec__max_features' : [15000],
    'cvec__ngram_range'  : [(1,3)]
}

# Model 5
model5 = nlp_model(X_train,X_test,y_train,y_test,'cvec','lr',params5)

**Model 6**

In [14]:
# Params 6
params6 = {
    'cvec__max_features' : [15000,20000],
    'cvec__ngram_range'  : [(1,3),(1,5)]
}

# Model 6
model6 = nlp_model(X_train,X_test,y_train,y_test,'cvec','lr',params6)

**Save to Scores List**

In [15]:
model1

,Model,Transform,Train Score,Test Score
0,lr,cvec,0.927339,0.906483


In [16]:
model2

,Model,Transform,Train Score,Test Score
0,lr,tvec,0.910987,0.903349


In [17]:
model3

,Model,Transform,Train Score,Test Score
0,lr,cvec,0.927339,0.906483


In [18]:
model4

,Model,Transform,Train Score,Test Score
0,lr,tvec,0.913101,0.903495


In [19]:
model5

,Model,Transform,Train Score,Test Score
0,lr,cvec,0.924764,0.906666


In [20]:
model6

,Model,Transform,Train Score,Test Score
0,lr,cvec,0.927339,0.906483


**Visualizing Coefs and ROC**

In [43]:
### BEST FIT

# Params 5
params5 = {
    'cvec__max_features' : [15000],
    'cvec__ngram_range'  : [(1,3)]
}

# Model 5
model5, model5_scores, model5_self = nlp_model(X_train,X_test,y_train,y_test,'cvec','lr',params5)

In [39]:
best_coefs = model5_scores['Coefficients']

In [ ]:
y_preds = 